# Improving the quality of white wine with neural networks
**Team**: Aaron Szabo

**Course**: CMSC 389A Practical Deep Learning  

**Description**: This program trains a model to predict the quality of white wine based on 11 charateristics. Then using the model it predicts what changes should be made to a given wine to maximally improve its quality. 

**Imports**  
This project requires keras, numpy, pandas, and sklearn.

In [1]:
import keras
import numpy as np
import pandas as pd

from keras.layers import Dense
from keras.models import Sequential
from sklearn.model_selection import train_test_split

C:\Users\Szabo\Anaconda3\envs\cmsc389\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load Data
Data is loaded from the white wine quality csv titled "winequality-white.csv" from _[UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/wine+quality)_<br>
Courtesy of:<br>
P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.<br>
Modeling wine preferences by data mining from physicochemical properties.<br>
In Decision Support Systems, Elsevier, 47(4):547-553. ISSN: 0167-9236.

In [2]:
file_name_white = 'winequality-white.csv'
columns = [
    'Fixed acidity',
    'Volatile acidity',
    'Citric acid',
    'Residual sugar',
    'Chlorides',
    'Free sulfur dioxide',
    'Total sulfur dioxide',
    'Density',
    'pH',
    'Sulphates',
    'Alcohol',
    'Quality'
]

df_white = pd.read_csv(file_name_white, names=columns, delimiter=';', header=0)
df_white[columns[:-1]] = df_white[columns[:-1]].astype(float)
df_white[columns[-1]] = df_white[columns[-1]].astype(float)

Normalizing 

In [3]:
# Normalize all the values between -0.5 and 0.5
for feature in df_white.columns[:-1]:
    max_value = df_white[feature].max()
    min_value = df_white[feature].min()
    mean_value = df_white[feature].mean()
    df_white[feature] = (df_white[feature] - mean_value) / (max_value - min_value)

Lets now examine the data after normalizing it.

In [4]:
df_white.head()

,Fixed acidity,Volatile acidity,Citric acid,Residual sugar,Chlorides,Free sulfur dioxide,Total sulfur dioxide,Density,pH,Sulphates,Alcohol,Quality
0,0.013963,-0.008080,0.015547,0.219457,-0.002292,0.033770,0.073409,0.134425,-0.171151,-0.046334,-0.276495,6.0
1,-0.053345,0.021332,0.003499,-0.073488,0.009578,-0.074244,-0.014758,-0.000528,0.101576,0.000178,-0.163591,6.0
2,0.119732,0.001724,0.039644,0.007800,0.012545,-0.018495,-0.095964,0.020679,0.065212,-0.057961,-0.066817,6.0
3,0.033193,-0.047295,-0.008549,0.032340,0.036284,0.040738,0.110532,0.030319,0.001576,-0.104473,-0.099075,6.0
4,0.033193,-0.047295,-0.008549,0.032340,0.036284,0.040738,0.110532,0.030319,0.001576,-0.104473,-0.099075,6.0


## Load the examples into a data variable
This is done for ease of use later in the program

In [5]:
class Example:
    def __init__(self, features, label):
        self.features = features
        self.label = label

In [6]:
data_white = []
for row in df_white.itertuples():
    label = row[-1]
    label = (label-3)/6
    features = row[1:-1]
    example = Example(features, label)
    data_white.append(example)

In [7]:
#data = shuffle(data, random_state=kSEED)

X_W = [example.features for example in data_white]
y_W = [example.label for example in data_white]

## Train-test Split

The data is split 80-20 train-test. 

In [8]:
X_train_W, X_test_W, y_train_W, y_test_W = train_test_split(X_W,y_W,test_size=0.2, random_state=10)

X_train_W = np.array(X_train_W)
y_train_W = np.array(y_train_W)
X_test_W = np.array(X_test_W)
y_test_W = np.array(y_test_W)

print('Total White Examples: {:}\nTrain Examples: {:}\nTest Examples: {:4d}'.format(len(data_white), len(X_train_W), len(X_test_W)))

Total White Examples: 4898
Train Examples: 3918
Test Examples:  980


## Implementing the Model



In [9]:
model = Sequential()
model.add(Dense(11,input_shape=(11,),activation='relu'))
model.add(Dense(20,activation='relu'))
model.add(Dense(30,activation='relu'))
model.add(Dense(40,activation='relu'))
model.add(Dense(30,activation='relu'))
model.add(Dense(20,activation='relu'))
model.add(Dense(11,activation='sigmoid'))
model.add(Dense(1,activation='sigmoid'))

### Compiling the Model

In [10]:
model.compile(loss='mean_squared_error', optimizer='adam')

### Model Summary

The model consists of 8 layers with sizes 1 to 40, see the below graph for the sizes of each layer. The first 6 layers use <code>relu</code> activation and the last two use <code>sigmoid</code> activation. This model was decided upon after extensive testing of various architectures. 

In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 11)                132       
_________________________________________________________________
dense_2 (Dense)              (None, 20)                240       
_________________________________________________________________
dense_3 (Dense)              (None, 30)                630       
_________________________________________________________________
dense_4 (Dense)              (None, 40)                1240      
_________________________________________________________________
dense_5 (Dense)              (None, 30)                1230      
_________________________________________________________________
dense_6 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_7 (Dense)              (None, 11)                231       
__________

### Training the Model

The model is trained on approximately all the data

In [24]:
model.fit(X_train_W, y_train_W, epochs=500, batch_size=10)

Epoch 1/500
3918/3918 [==============================] - 0s 79us/step - loss: 0.0032
Epoch 2/500
3918/3918 [==============================] - 0s 82us/step - loss: 0.0030
Epoch 3/500
3918/3918 [==============================] - 0s 82us/step - loss: 0.0032
Epoch 4/500
3918/3918 [==============================] - 0s 81us/step - loss: 0.0030
Epoch 5/500
3918/3918 [==============================] - 0s 81us/step - loss: 0.0029
Epoch 6/500
3918/3918 [==============================] - 0s 80us/step - loss: 0.0031
Epoch 7/500
3918/3918 [==============================] - 0s 82us/step - loss: 0.0029
Epoch 8/500
3918/3918 [==============================] - 0s 83us/step - loss: 0.0031
Epoch 9/500
3918/3918 [==============================] - 0s 81us/step - loss: 0.0029
Epoch 10/500
3918/3918 [==============================] - 0s 84us/step - loss: 0.0034
Epoch 11/500
3918/3918 [==============================] - 0s 84us/step - loss: 0.0031
Epoch 12/500
3918/3918 [==============================] - 0s 83

3918/3918 [==============================] - 0s 82us/step - loss: 0.0025
Epoch 97/500
3918/3918 [==============================] - 0s 89us/step - loss: 0.0026
Epoch 98/500
3918/3918 [==============================] - 0s 85us/step - loss: 0.0028
Epoch 99/500
3918/3918 [==============================] - 0s 83us/step - loss: 0.0030
Epoch 100/500
3918/3918 [==============================] - 0s 82us/step - loss: 0.0026
Epoch 101/500
3918/3918 [==============================] - 0s 100us/step - loss: 0.0028
Epoch 102/500
3918/3918 [==============================] - 0s 113us/step - loss: 0.0032
Epoch 103/500
3918/3918 [==============================] - 0s 111us/step - loss: 0.0026
Epoch 104/500
3918/3918 [==============================] - 0s 102us/step - loss: 0.0027
Epoch 105/500
3918/3918 [==============================] - 0s 96us/step - loss: 0.0025
Epoch 106/500
3918/3918 [==============================] - 0s 93us/step - loss: 0.0028
Epoch 107/500
3918/3918 [==============================]

3918/3918 [==============================] - 0s 96us/step - loss: 0.0023
Epoch 190/500
3918/3918 [==============================] - 0s 80us/step - loss: 0.0023
Epoch 191/500
3918/3918 [==============================] - 0s 96us/step - loss: 0.0024
Epoch 192/500
3918/3918 [==============================] - 0s 126us/step - loss: 0.0022
Epoch 193/500
3918/3918 [==============================] - 0s 97us/step - loss: 0.0024
Epoch 194/500
3918/3918 [==============================] - 0s 96us/step - loss: 0.0024
Epoch 195/500
3918/3918 [==============================] - 0s 81us/step - loss: 0.0023
Epoch 196/500
3918/3918 [==============================] - 0s 80us/step - loss: 0.0027
Epoch 197/500
3918/3918 [==============================] - 0s 89us/step - loss: 0.0033
Epoch 198/500
3918/3918 [==============================] - 0s 84us/step - loss: 0.0024
Epoch 199/500
3918/3918 [==============================] - 0s 89us/step - loss: 0.0023
Epoch 200/500
3918/3918 [==============================]

3918/3918 [==============================] - 0s 81us/step - loss: 0.0024
Epoch 284/500
3918/3918 [==============================] - 0s 77us/step - loss: 0.0022
Epoch 285/500
3918/3918 [==============================] - 0s 79us/step - loss: 0.0024
Epoch 286/500
3918/3918 [==============================] - 0s 83us/step - loss: 0.0021
Epoch 287/500
3918/3918 [==============================] - 0s 82us/step - loss: 0.0022A: 0s - loss: 0.00
Epoch 288/500
3918/3918 [==============================] - 0s 85us/step - loss: 0.0020
Epoch 289/500
3918/3918 [==============================] - 0s 78us/step - loss: 0.0020
Epoch 290/500
3918/3918 [==============================] - 0s 94us/step - loss: 0.0021
Epoch 291/500
3918/3918 [==============================] - 0s 78us/step - loss: 0.0023
Epoch 292/500
3918/3918 [==============================] - 0s 86us/step - loss: 0.0022
Epoch 293/500
3918/3918 [==============================] - 0s 85us/step - loss: 0.0023
Epoch 294/500
3918/3918 [==============

3918/3918 [==============================] - 0s 82us/step - loss: 0.0024
Epoch 378/500
3918/3918 [==============================] - 0s 88us/step - loss: 0.0022
Epoch 379/500
3918/3918 [==============================] - 0s 83us/step - loss: 0.0022
Epoch 380/500
3918/3918 [==============================] - 0s 96us/step - loss: 0.0020
Epoch 381/500
3918/3918 [==============================] - 0s 95us/step - loss: 0.0020
Epoch 382/500
3918/3918 [==============================] - 0s 78us/step - loss: 0.0019
Epoch 383/500
3918/3918 [==============================] - 0s 94us/step - loss: 0.0017
Epoch 384/500
3918/3918 [==============================] - 0s 98us/step - loss: 0.0021
Epoch 385/500
3918/3918 [==============================] - 0s 108us/step - loss: 0.0027
Epoch 386/500
3918/3918 [==============================] - 0s 92us/step - loss: 0.0019
Epoch 387/500
3918/3918 [==============================] - 0s 101us/step - loss: 0.0020
Epoch 388/500
3918/3918 [==============================

3918/3918 [==============================] - 0s 85us/step - loss: 0.0019
Epoch 472/500
3918/3918 [==============================] - 0s 76us/step - loss: 0.0021
Epoch 473/500
3918/3918 [==============================] - 0s 75us/step - loss: 0.0019
Epoch 474/500
3918/3918 [==============================] - 0s 75us/step - loss: 0.0021
Epoch 475/500
3918/3918 [==============================] - 0s 75us/step - loss: 0.0021
Epoch 476/500
3918/3918 [==============================] - 0s 74us/step - loss: 0.0016
Epoch 477/500
3918/3918 [==============================] - 0s 75us/step - loss: 0.0018
Epoch 478/500
3918/3918 [==============================] - 0s 74us/step - loss: 0.0019
Epoch 479/500
3918/3918 [==============================] - 0s 74us/step - loss: 0.0022
Epoch 480/500
3918/3918 [==============================] - 0s 74us/step - loss: 0.0019
Epoch 481/500
3918/3918 [==============================] - 0s 74us/step - loss: 0.0018
Epoch 482/500
3918/3918 [==============================] 

In [25]:
print("Done")

Done


### Evaluating the Model

<code>rng</code> sets the range around the correct value which will be considered as correct. Specifically if $| pred - label | < 1 /rng$ then that data point will be considered correct.

In [14]:
def accu_check(model, test_data, test_pred, bound):
    y_pred = model.predict(test_data)

    total = 0
    right = 0.0
    for (pred,test) in zip(y_pred,test_pred):
        total += 1
        if pred >= test - bound/2 and pred <= test + bound/2:
            right += 1
    accuracy = right/total
    print("Total elements checked: {}".format(total))
    print("Correct predictions made: {}".format(int(right)))
    print("Accuracy: {:.4}".format(accuracy))

In [22]:
rng = 6
accu_check(model, X_test_W, y_test_W, 1.0/rng)

Total elements checked: 980
Correct predictions made: 600
Accuracy: 0.6122


Typically the model has an accuracy of $>0.7$ with <code>rng = 4</code> and $>0.6$ when <code>rng = 6</code>

## Application

The program takes the test case given below and finds the optimal changes to maximally improve the quality of the selected wine.

In [17]:
print("Enter a value between 1 and {:4d}".format(len(X_train_W)))
index = input(":")
try:
    index = int(index)
    if index < 1 or index > len(X_train_W):
        print("Error in input")
    else:
        index -= 1
except ValueError:
    print("Error in input")

Enter a value between 1 and 3918
:6
6


In [18]:
#Don't run this cell if the above cell prints an error
vary_range = 0.2
def adjust(val, delta):
    if delta == 0:
        val = val - vary_range
        if val < 0: val = 0
    elif delta == 2:
        val = val + vary_range
        if val > 0: val = 1
    return val

In [19]:
vary_range = 0.2
opt = np.zeros((3,3,3,3,3,3,3,3,3,3,3), np.float32)
start = X_test_W[index]
for a in range(3):
    for b in range(3):
        for c in range(3):
            for d in range(3):
                for e in range(3):
                    for f in range(3):
                        for g in range(3):
                            for h in range(3):
                                for i in range(3):
                                    for j in range(3):
                                        for k in range(3):
                                            temp = np.array([np.zeros((11,), np.float32)])
                                            temp[0] = np.array([adjust(start[0], a),adjust(start[1], b),adjust(start[2], c),adjust(start[3], d),adjust(start[4], e),adjust(start[5], f),adjust(start[6], g),adjust(start[7], h),adjust(start[8], i),adjust(start[9], j),adjust(start[10], k)])
                                            opt[a,b,c,d,e,f,g,h,i,j,k] = model.predict(temp)
print("Done")

Done


In [20]:
def translate(val):
    if val == 0:
        return "Decrease"
    elif val == 1:
        return "Don't change"
    else:
        return "Increase"

In [21]:
ind = np.unravel_index(np.argmax(opt, axis=None), opt.shape)
print("{} fixed acidity\n{} volatile acidity\n{} citric acid\n{} residual sugar\n{} chlorides\n{} free sulfur dioxide\n{} total sulfur dioxide\n{} density\n{} pH\n{} sulphates\n{} alcohol".format(translate(ind[0]),translate(ind[1]),translate(ind[2]),translate(ind[3]),translate(ind[4]),translate(ind[5]),translate(ind[6]),translate(ind[7]),translate(ind[8]),translate(ind[9]),translate(ind[10])))

Increase fixed acidity
Decrease volatile acidity
Increase citric acid
Increase residual sugar
Decrease chlorides
Increase free sulfur dioxide
Decrease total sulfur dioxide
Decrease density
Decrease pH
Increase sulphates
Increase alcohol
